In [1]:
import json
from ase import Atoms, Atom
from ase.visualize import view
from ase.constraints import dict2constraint
from ase.calculators.singlepoint import SinglePointCalculator

In [2]:
with open('co_data.json', 'r') as file_handle:
    co_documents = json.load(file_handle)

In [3]:
co_documents[0]

{'adsorbate': 'CO',
 'mpid': 'mp-579544',
 'miller': [1, 1, 0],
 'shift': 0.2614824625000001,
 'top': True,
 'coordination': 'Cu',
 'neighborcoord': ['Cu:Ca-Ca-Sn-Sn-Sn'],
 'energy': -0.5730929899999975,
 'atoms': {'atoms': [{'symbol': 'C',
    'position': [4.950828302364764, 7.456137729876762, 20.64137890207189],
    'tag': 1,
    'index': 0,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmom': 0.0},
   {'symbol': 'O',
    'position': [4.937282334162113, 7.478494685664874, 21.81204637521394],
    'tag': 1,
    'index': 1,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmom': 0.0},
   {'symbol': 'Ca',
    'position': [4.62461883579917, 8.797517786901787, 11.609944634852008],
    'tag': 0,
    'index': 2,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmom': 0.0},
   {'symbol': 'Ca',
    'position': [6.645185404401976, 3.6846857888129585, 14.482524119130769],
    'tag': 0,
    'index': 3,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmo

In [4]:
def make_atoms_from_doc(doc):
    '''
    This is the inversion function for `make_doc_from_atoms`; it takes
    Mongo documents created by that function and turns them back into
    an ase.Atoms object.

    Args:
        doc     Dictionary/json/Mongo document created by the
                `make_doc_from_atoms` function.
    Returns:
        atoms   ase.Atoms object with an ase.SinglePointCalculator attached
    '''
    atoms = Atoms([Atom(atom['symbol'],
                        atom['position'],
                        tag=atom['tag'],
                        momentum=atom['momentum'],
                        magmom=atom['magmom'],
                        charge=atom['charge'])
                   for atom in doc['atoms']['atoms']],
                  cell=doc['atoms']['cell'],
                  pbc=doc['atoms']['pbc'],
                  info=doc['atoms']['info'],
                  constraint=[dict2constraint(constraint_dict)
                              for constraint_dict in doc['atoms']['constraints']])
    results = doc['results']
    calc = SinglePointCalculator(energy=results.get('energy', None),
                                 forces=results.get('forces', None),
                                 stress=results.get('stress', None),
                                 atoms=atoms)
    atoms.set_calculator(calc)
    return atoms

In [5]:
initial_atoms = make_atoms_from_doc(co_documents[0]['initial_configuration'])
view(initial_atoms, viewer='x3d')

In [6]:
relaxed_atoms = make_atoms_from_doc(co_documents[0])
view(relaxed_atoms, viewer='x3d')